In [ ]:
from typing import Literal, Callable, Any
import math, time, json, random
from random import randint

from drivingppo.common import set_seed, SPD_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.environment import speed_norm, get_pp_lookahead_point
from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.simsim import WorldViewer

from world_samples import NEAR, gen_0, gen_1, gen_2, gen_2l, generate_random_world_plain, gen_obs

import numpy as np

W_CONFIG = {
    'map_border': False,
    'near': NEAR,
    'far': 999.9,
}


class PurePursuitController:
    def __init__(self, target_speed=SPD_SCFAC, min_ld=1.0, ld_k=1.0):
        """
        :param target_speed: 목표 주행 속도 (m/s)
        :param min_ld: 최소 전방 주시 거리 (m)
        :param ld_k: 속도 비례 전방 주시 계수 (Ld = min_ld + ld_k * speed)
        """
        self.target_speed = target_speed
        self.min_ld = min_ld
        self.ld_k = ld_k
        self.kp_steer = 1.5  # 조향 비례 제어 게인

    def get_action(self, world:World) -> tuple[float, float, bool]:
        """
        World 개체를 받아 액션(ws, ad, stop) 반환
        """
        p = world.player
        px, pz = p.x, p.z

        # 이미 도착한 경우 정지
        if world.arrived:
            return 0.0, 0.0, True

        ld = self.min_ld + self.ld_k * max(0, p.speed)
        lookahead_point, _ = get_pp_lookahead_point(world, ld)

        # 조향 제어 (ad)
        tx, tz = lookahead_point
        abs_angle = angle_of(px, pz, tx, tz)

        rel_angle = abs_angle - p.angle_x
        rel_angle = (rel_angle + math.pi) % (2*math.pi) - math.pi  # 정규화: -pi ~ pi

        ad = self.kp_steer * rel_angle
        ad = max(min(ad, 1.0), -1.0)  # 액션 유효범위로 클리핑

        # 종방향 제어 (ws)
        current_target_speed = self.target_speed * (1.0 - ad*ad * 0.6)  # 곡선 구간(조향각이 클 때)에서는 속도를 줄이기

        speed_error = current_target_speed - p.speed
        ws = speed_error * 0.5  # 가속도 비례 게인
        ws = max(min(ws, 1.0), -1.0)

        return ws, ad, False


def testew():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 50),
            (20, 50),
            (50, 70),
            (60, 60),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w


def run(
        world_generator:Callable[[], World],
        con:PurePursuitController,
        time_spd=2.0,
        time_step=WORLD_DT,
        action_repeat=ACTION_REPEAT,
    ):

    world = world_generator()
    viewer = WorldViewer(world, auto_update=False)

    # 지표 수집용 변수 초기화
    estep_count = 0
    ws_penalty_sum = 0.0
    ad_penalty_sum = 0.0
    speed_list = []

    while not world.arrived:

        estep_count += 1
        ws, ad, _ = con.get_action(world)  # 컨트롤러가 조작 산출

        s_norm = speed_norm(world.player.speed)
        ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0
        ws_penalty_sum += ws_penalty
        ad_penalty_sum += (- ad * ad * 1.7)
        speed_list.append(world.player.speed)

        for _ in range(action_repeat):
            world.set_action(ws, ad)  # 행동 실행
            world.step(time_step)

        if viewer.closed: break
        viewer.update()  # 시각화 호출
        time.sleep(time_step / 1000.0 / time_spd)  # 시각화 프레임을 위해 딜레이 추가

    tcount = estep_count * action_repeat * time_step / 1000.0  # 흐른 시간 (초)
    final_ws = ws_penalty_sum / tcount
    final_ad = ad_penalty_sum / tcount
    speed_mean = np.mean(speed_list)  if speed_list else 0.0
    speed_var  = np.var(speed_list)   if speed_list else 0.0

    print("\n=== Pure Pursuit 에피소드 종료 지표 ===")
    print(f"rewards/6.ws      : {final_ws:.4f}")
    print(f"rewards/7.ad      : {final_ad:.4f}")
    print(f"metrics/speed_mean: {speed_mean:.4f}")
    print(f"metrics/speed_var : {speed_var:.4f}")
    print("=======================================\n")

    viewer.occupy_mainloop()

def gen_0():  return generate_random_world_plain(map_h= 50, map_w= 50, num=1,                min_dist=10,  max_dist=10, ang_init='half', ang_lim=0,      spd_init=0)
def gen_1():  return generate_random_world_plain(map_h=150, map_w=150, num=3,                min_dist=10,  max_dist=20, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')
def gen_2():  return generate_random_world_plain(map_h=150, map_w=150, num=LOOKAHEAD_POINTS, min_dist=10,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')
def gen_2l(): return generate_random_world_plain(map_h=150, map_w=150, num=10,               min_dist=10,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')

con = PurePursuitController()
# set_seed(1)
run(gen_2l, con)
